# 🎨 Image Augmentation

Data preparation is a crucial task when working with neural networks and deep learning models. Therefore, when preparing data, data augmentation is used to increase the amount of data. In other words, having a large dataset allows you to have a larger dataset which in return gives you a more robust model. 

However acquiring more data is not easy and there can be problems of storing them and feeding it to the model. To mitigate this issue, we can either increase the data manually be doing some changes or we can use one of Keras image pre-processing classes to increase the number of images.

## 🗂️ Import Libraries
The following code block imports all the libraries that are necessary for our development purposes.

#### 🧠 `keras`
- Keras is a high-level deep learning library in Python that simplifies the creation and training of neural networks by providing an intuitive interface to underlying frameworks like TensorFlow.

#### ⚡ `tensorflow`
- TensorFlow is an open-source library designed for building and deploying machine learning and deep learning models, offering a flexible ecosystem for numerical computation and AI development.

#### 💻 `os`
- The os library in Python provides a way to interact with the operating system, enabling tasks such as file and directory manipulation, environment variable access, and process management.

#### 📄 `shutil`
- The shutil library in Python provides a suite of functions for high-level file operations, including copying, moving, renaming, and deleting files and directories.

In [1]:
## 🗂️ Import libraries

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img, img_to_array, load_img
import os
import shutil

## ♾️ Constants
While proceeding with image augementation process, we need some constants that may refer the directory paths, image category names, data generation properties, etc.

In [2]:
## ♾️ Constants

### Directory Paths
ORIGINAL_DATA_PATH = './data/original'
AUGMENTED_DATA_PATH = './data/augmented'
TRAIN_DIRECTORY = 'train'
TEST_DIRECTORY = 'test'
VALIDATION_DIRECTORY = 'val'

### Directories containing the images
SUB_DIRECTORIES = [TRAIN_DIRECTORY, TEST_DIRECTORY, VALIDATION_DIRECTORY]
DATA_DIRECTORIES = ['Actinic keratoses', 'Basal cell carcinoma', 'Benign keratosis-like lesions', 'Chickenpox', 'Cowpox', 'Dermatofibroma', 'Healthy', 'HFMD', 'Measles', 'Melanocytic nevi', 'Melanoma', 'Monkeypox', 'Squamous cell carcinoma', 'Vascular lesions']

### Specific files
DS_STORE = '.DS_Store'

### Data generation properties
ROTATION_RANGE = 40
WIDTH_SHIFT_RANGE = 0.2
HEIGHT_SHIFT_RANGE = 0.2
SHEAR_RANGE = 0.2
ZOOM_RANGE = 0.2
HORIZONTAL_FLIP = True
FILL_MODE = 'nearest'

## 💾 Define Data Generation Properties
Next, we need to define data-gen properties. This way more data is generated according to the parameters we define. The parameters we will be using for this are as follows.

- ↻ `rotation_range`
    - Specifies the range(in degrees) within which the image is randomly rotated.
    - For example, a value of `40` means the image can be rotated randomly by up to `40` degrees in either direction(clockwise or counterclockwise).<br><br>
    
- ↔ `width_shift_range`
    - Specifies the fraction of the total width of the image by which it can be randomly shifted horizontally.
    - For example, a value of `0.2` allows a horizontal shift of up to `20%` of the image's width.<br><br>
    
- ↕ `height_shift_range`
    - Specifies the fraction of the total height of the image by which it can be randomly shifted vertically.
    - For example, a value of `0.2` allows a vertical shift of up to `20%` of the image's height.<br><br>
  
- ➤ `shear_range`
    - Specifies the intensity of shear transformation as a shear angle in a counterclockwise direction in degrees.
    - A shear transformation distorts the image along an axis creating a parallelogram effect.<br><br>

- 🔎 `zoom_range`
   - Specifies the range for random zoom. It can zoom in or out within this range.
   - For example, a value of `0.2` means the image size can be varied randomly by up to `±20%`.<br><br>

- ― `horizontal_flip`
   - Specifies whether the image should be flipped horizontally.
   - If `True`, the image is randomly flipped along the horizontal axis, mirroring it.<br><br>

- 🚰 `fill_mode`
   - Specifies the strategy for filling in pixels that are introduced during transformations.
   - The value `nearest` means the nearest pixel values are used to fill these gaps.

In [3]:
## 💾 Define Data Generation Properties

datagen = ImageDataGenerator(
    rotation_range=ROTATION_RANGE,
    width_shift_range=WIDTH_SHIFT_RANGE,
    height_shift_range=HEIGHT_SHIFT_RANGE,
    shear_range=SHEAR_RANGE,
    zoom_range=ZOOM_RANGE,
    horizontal_flip=HORIZONTAL_FLIP,
    fill_mode=FILL_MODE)

## 📂 Create Directories to Store the Augmented Images

Since our original data is presented in the following way, we need to create augmented images for all, `train`, `test`, and `val`(`validation`) directories. 

```
data
 └── original
       ├── test
       ├── train
       └── val
```

Therefore, we need to have a separate directory to hold the augmented images.

```
data
 └── augmented
       ├── test
       ├── train
       └── val
 └── original
       ├── test
       ├── train
       └── val
```


Also, we will only be using the augmented images for training our models. However, image augmentation is a unilateral task from training the models. Therefore, having all the images augmented in one go will make our development easier.

In [4]:
## 📂 Create Directories to Store the Augmented Images

### Create train, test, and val directories
for sub_directory in SUB_DIRECTORIES:
    path = AUGMENTED_DATA_PATH
    path = os.path.join(path, sub_directory)
    os.mkdir(path)

### Create data directories inside of the train, test, and val directories
for data_directory in DATA_DIRECTORIES:
    for sub_directory in SUB_DIRECTORIES:
        path = AUGMENTED_DATA_PATH
        path = os.path.join(path, sub_directory, data_directory)
        os.mkdir(path)

## ✨ Generate Images

Now we need to iterates through the data directories and subdirectories, applies data augmentation to images, and saves the augmented images alongside the originals in a specified output directory. 

For each image file, we need to also skip the hidden system files like `.DS_Store`, loads the image using `load_img`, converts it to a NumPy array using `img_to_array`, and reshapes it to a 4D array suitable for batch processing. 

Afterwards, using the data generator we defined, we can create batches of augmented images by applying random transformations like rotation, zoom, and flipping, saving up to 20 transformed versions of each image in the output directory. 

Finally, we copy the original image to that output directory as well.

In [5]:
## ✨ Generate Images

for data_directory in DATA_DIRECTORIES:
    for sub_directory in SUB_DIRECTORIES:
        path_o = os.path.join(ORIGINAL_DATA_PATH, sub_directory, data_directory)
        path_a = os.path.join(AUGMENTED_DATA_PATH, sub_directory, data_directory)
        files = os.listdir(path_o)
        for index, file in enumerate(files):
            if DS_STORE in file: ### Skip .DS_Store file
                continue
            print(f'{path_o}/{file}') ### Print the pathname and file name to get an idea about the processing file
            img = load_img(str(os.path.join(path_o, file))) ### This is a PIL image
            x = img_to_array(img) ### This is a NumPy array with shape (3, 150, 150)
            x = x.reshape((1,) + x.shape) ### This is a NumPy array with shape (1, 3, 150, 150)
            ### The .flow() command generates batches of randomly transformed images and save the results in the ./data/augmented/<sub_directory>/<data_directory> along with the original image
            i = 0
            for batch in datagen.flow(x, batch_size=1, save_to_dir=path_a, save_prefix=file.split(".")[0], save_format='jpg'):
                i += 1
                if i > 20:
                    break ### Stop looping the generator infinitely
            ### Save the original image in ./data/augmented/<sub_directory>/<data_directory> as well
            shutil.copy(os.path.join(path_o, file), path_a)    

./data/augmented/val/Vascular lesions/ISIC_0062878.jpg
./data/augmented/val/Vascular lesions/ISIC_0069939.jpg
./data/augmented/val/Vascular lesions/ISIC_0071186.jpg
./data/augmented/val/Vascular lesions/ISIC_0060706.jpg
./data/augmented/val/Vascular lesions/ISIC_0029460.jpg
./data/augmented/val/Vascular lesions/ISIC_0065118.jpg
./data/augmented/val/Vascular lesions/ISIC_0064951.jpg
./data/augmented/val/Vascular lesions/ISIC_0064789.jpg
./data/augmented/val/Vascular lesions/ISIC_0059408.jpg
./data/augmented/val/Vascular lesions/ISIC_0053657.jpg
./data/augmented/val/Vascular lesions/ISIC_0057431.jpg
./data/augmented/val/Vascular lesions/ISIC_0063019.jpg
./data/augmented/val/Vascular lesions/ISIC_0067519.jpg
./data/augmented/val/Vascular lesions/ISIC_0031993.jpg
./data/augmented/val/Vascular lesions/ISIC_0056920.jpg
./data/augmented/val/Vascular lesions/ISIC_0061154.jpg
./data/augmented/val/Vascular lesions/ISIC_0056115.jpg
./data/augmented/val/Vascular lesions/ISIC_0059540.jpg
./data/aug

KeyboardInterrupt: 

## 📚 References

1. [Prachi Gopalani Medium Blog](https://medium.com/analytics-vidhya/image-augmentation-using-keras-99072b490c72#:~:text=Keras%20Image%20Augmentation%20API%3A%20ImageDataGenerator,a%20new%20batch%20for%20training.)
2. [Keras Blog](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)